In [1]:
from composable import pipeable
from toolz.curried.operator import add
from toolz.curried.operator import eq
from composable.strict import map
from toolz.curried import get
import composable.strict as s
import re

In [2]:
with open('sell_short_trades.txt') as f:
    lines = f.readlines()
lines[:10]
no_header = lines[7:]


In [3]:
stripped_lines = [line.strip() for line in no_header if line.strip()]
stripped_lines[:20]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866096       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  869780       23',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX',

In [57]:
header = re.compile('Symbol|TradeCommType')
page_num = re.compile('\d of \d\d')
extra_num = re.compile('854300')
is_page_num_line = lambda l: page_num.search(l)
is_blank = lambda l: len(l.strip()) == 0
is_extra = lambda l: extra_num.search(l)
is_header = lambda l: header.search(l)
needs_removal = lambda l: is_blank(l) or is_page_num_line(l) or is_extra(l) or is_header(l)
culled_lines = [l for l in stripped_lines if not needs_removal(l)]
culled_lines[:10]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866096       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX']

In [58]:
get_layer = lambda l: l[:3]
joined = ','.join(get_layer(stripped_lines))
joined


'CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS,/17  /17  256945       00,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO'

In [59]:
stop_condition = lambda s: len(s) <= 3
get_layer = lambda l: l[:3]
comma_join = lambda l: ','.join(get_layer(l))
get_remaining = lambda l: l[3:]
new_seq = []
remaining_layers = culled_lines
while not stop_condition(remaining_layers):   
    new_seq = new_seq + [comma_join(get_layer(remaining_layers))]
    remaining_layers = get_remaining(remaining_layers)
new_seq[:10]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS,/17  /17  256945       00,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866087       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  866096       04,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX,/17  /17  869780       23,0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX,/17  /17  892888       29,0704 PerUnit   TERM  0.010000     AABBC 

In [60]:
stripped = [line.strip().split() for line in new_seq]
stripped[:5]
joined = '\n'.join([','.join(line) for line in stripped])
joined[:500]


'CERS,SS,NEW,2,756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO\nCERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS\nCERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS\nCERS,SS,NEW,100,2.350000'

In [61]:
with open('sell_short_trades_Grouped.csv','w') as outfile:
    outfile.write(joined)

!cat 911_Deaths_Grouped.csv | head -n 5

cat: 911_Deaths_Grouped.csv: No such file or directory


In [62]:
with open('sell_short_trades_Grouped.csv') as f:
    content = f.read()
g_lines = content.split('\n')
g_lines[:10]

['CERS,SS,NEW,2,756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO',
 'CERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,869780,23,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:22:ContraSSFREX,/17,/17,892888,29,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.360000,2.350000,2.36000006/1406/191706149900003,10:24:ContraSSFREX,/17,/17,909119,01,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS',
 'CERS,SS,NEW,100,2.360

In [64]:
expression = re.compile('(CERS|S|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]
                        

[]

In [67]:
expression = re.compile('.*,(SS|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]

[]

In [ ]:
expression = re.compile('.*,(SS|/17),.*')
[(i, l) for i, l in enumerate(g_lines) if not expression.search(l)]

In [81]:
expression1 = re.compile('(CERS|S|/17),(SS|/17),(NEW|\d\d\d\d\d\d),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*),(/17,/17|0704),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*)')
expression2 = re.compile('(/17),(/17),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*)')
expression3 = re.compile('(CERS|S),(SS),(NEW),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=.*),(\d\d\d\d\d\d),(\d\d),(0704),(PerUnit),(TERM),(\d\.\d*),(AABBC),(\d*#STA=),(/17,/17|0704),(\d\d\d\d\d\d),(\d\d),(CERS|S),(SS),(NEW),(\d\,\d\d\d|\d00),(\d\.\d*\,\d\.\d*|\d\.\d*\d\.\d*),(\d\.\d*/1406/\d*),(\d\d:\d\d:.*)')

in_file = lambda l: expression1.search(l) or expression2.search(l) or expression3.search(l)
culled_lines = [l for l in stripped_lines if not needs_removal(l)]
[(i, l) for i, l in enumerate(g_lines) if not in_file]
                        

[]